In [ ]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [ ]:
#  Add the clean covid function that takes in the argument, "covid".
def clean_covid_data(covid):
    covid = dict(covid) #create a non-destructive copy

# merge column names
    def change_column_name(old_name, new_name):
        if old_covid in covid:
            covid[new_name] = covid.pop(old_name)
    change_column_name('Fips', 'FipCode')
    change_column_name('metrics.testPositivityRatio', 'PositivityRatio')
    change_column_name('metrics.caseDensity', 'Density')
    change_column_name('metrics.infectionRatio', 'InfectionRatio')
    change_column_name('metrics.infectionRatioCI90', 'InfectionRateCI90')
    change_column_name('metrics.icuHeadroomRatio', 'IcuHeadRoomRatio')
    change_column_name('metrics.icuCapacityRatio', 'IcuCapacityRatio')
    change_column_name('riskLevels.overall', 'RiskLevelOverall')
    change_column_name('riskLevels.testPositivityRatio', 'RiskLevelPositivityRatio')
    change_column_name('riskLevels.caseDensity', 'RiskLevelCaseDensity')
    change_column_name('riskLevels.contactTracerCapacityRatio', 'RiskLevelContactTracerCapacityRatio')
    change_column_name('riskLevels.infectionRate', 'RiskLevelInfectionRate')
    change_column_name('riskLevels.icuHeadroomRatio', 'RiskLevelICUHeadRoomRatio')
    change_column_name('riskLevels.icuCapacityRatio', 'RiskLevelsICUCapacityRatio')
    change_column_name('Actuals.cases', 'ActualCases')
    change_column_name('Actuals.deaths', 'ActualDeaths')
    change_column_name('actuals.hospitalBeds.capacity', 'ActualHospitalBedCapacity')
    change_column_name('actuals.hospitalBeds.currentUsageTotal', 'ActualHospitalBedCurrentUsageTotal')
    change_column_name('actuals.hospitalBeds.currentUsageCovid', 'ActualHospitalBedCurrentUsageCovid')
    change_column_name('actuals.hospitalBeds.typicalUsageRate', 'ActualHospitalBedTypicalUsageRate')
    change_column_name('actuals.icuBeds.capacity', 'ActualIcuBedsCapacity')
    change_column_name('actuals.icuBeds.currentUsageTotal', 'ActualIcuBedCurrentUsageTotal')
    change_column_name('actuals.icuBeds.currentUsageCovid', 'ActualIcuBedCurrentUsageCovid')
    change_column_name('actuals.icuBeds.typicalUsageRate', 'ActualIcuBedTypicalUsageRate')
    change_column_name('actuals.newCases', 'ActualNewCases')
    change_column_name('actuals.vaccinationsInitiated', 'ActualVaccinationInitiated')
    change_column_name('actuals.vaccinationsCompleted', 'ActualVaccinationCompleted')
    change_column_name('metrics.vaccinationsInitiatedRatio', 'VaccinationInitaitedRatio')
    change_column_name('metrics.vaccinationsCompletedRatio', 'VaccinationCompletedRatio')
    change_column_name('actuals.newDeaths', 'ActualNewDeaths')
    change_column_name('actuals.vaccinesAdministered', 'ActualVaccinesAdministered')


    return covid

In [ ]:
# 1 Add the function that takes in one argument;
# covid data from apidocs.covidactnow.org

def extract_transform_load(covid, epa):
    # Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    # kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    # ratings = pd.read_csv(ratings_file, low_memory=False)

    # Open and read the Wikipedia data JSON file.
    with open(covid_file, mode='r') as file:
        covid_raw = json.load(file)


    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    clean_covid = [clean_covid(covid) for covid in wiki_movies]


In [ ]:
# Path to your file directory and variables for the three files.
file_dir = "/Users/joepedroza/Desktop/Data Visualization Class/Analysis Projects/UT-VIRT-DATA-GROUP-6-PROJECT/Resources"
# COVID data by state
covid_by_state_file = f'{file_dir}/covid_by_state.json'
# COVID data by state series
covid_by_state_series_file = f'{file_dir}/covid_by_state_series.json'
# COVID data by state for Texas
covid_by_texas_file= f'{file_dir}/TX.csv'
# COVID data by state by date series for Texas
covid_by_texas_file= f'{file_dir}/TX.timeseries.csv'

In [ ]:
# Set the variables equal to the extract_transform_load function.
covid_texas_file = extract_transform_load(covid_by_texas_file)